# Recurrent Neural Networks
So far, the neural network architectures we have been using take in a single fixed size input and give a single fixed size output. What if we wanted to model something like language where we want to feed in different length words or sentences? Another issue is that each output is only dependent on the current input. It has no 'memory' of previous inputs so you can't model time dependent variables. Recurrent neural networks address both these issues.

They do this by having an internal hidden state which can be thought of as a form of memory. At each time step, the new hidden state is calculated as a function of the previous hidden state and the current input. This hidden state can then be used to represent your output or can be put through another function to compute the outputs. When we say function we are referring to the same one used in standard neural network: linear combination followed by an activation function.

### $h_t = f(x_t, h_{t-1})$

As shown in the diagram below, which uses a further function to compute the output $o$ from the hidden state $s$, there are three matrices of parameters which we are trying to optimize: U, V and W. The diagram also demonstrates how these networks can be unfolded to show the variables at various time steps.

![image](images/rnn.jpg)

Standard neural networks can only model one to one relationships while RNNs are extremely flexible in terms of input-output structures which is one of the reasons they are so powerful. You can imagine something like one to many being used to feed in a single image from which a caption is sequentially produced or a many to one being used to feed in a sentence sequentially and give a single output describing the sentiment of the sentence.

![image](images/rnnlayouts.jpeg)

### Optimization
Surprisingly, with this increased complexity in structure, the optimization method does not become any more difficult. Despite having a different name, back-propagation through time, it is essentially the same thing. All you do is feed in your sequence sequentially to get the output, as usual. You then just calculate your error at each timestep and sum it as opposed to calculating the error at a single timestep like standard neural networks. Then you can use gradient descent to update your weights iteratively until you are satisfied with your network's performance.

RNNS are generally slower to optimize than standard neural networks as the output at each time step is dependent on the previous output so the operations cannot be parallelized.

For a long time it was considered difficult to train RNNs due to two problems called vanishing and exploding gradients. These problems also exist in standard neural network but are greatly emphasized in RNNs. However, modern techniques such as LSTM cells have greatly reduced this difficulty.

For this particular task, we will need to do quite a bit of pre-processing. We need to find the number of unique characters in our training text and give each one a unique number so we can one-hot encode them.<br>
We start by reading the file, converting all letters to lowercase to reduce the number of characters we need to model, then defining a function which takes in the text and gives up back a dictionary mapping each letter to a unique number.

## Implementation
We are going to be implementing a one-to-one character level text prediction model. We will be sequentially feeding in a single character and asking our network to predict the next character as a time dependent function of all the characters that came before it.

As always, we begin by importing the required libraries.

In [1]:
import torch
import numpy as np
import torch.nn.functional as F

Lets read in our dataset. We just need a text file which contains the data which we want to model. In this case, I have found a file which contains ~0.5MB of Kendrick Lamar lyrics. You can use a variety of different datasets. There are plenty of which are easily accessible online - check out the links below. Otherwise, is very easy to create your own either by copying and pasting text into a file or creating a bot to automatically do this for you.

[Datasets repo 1](https://github.com/cedricdeboom/character-level-rnn-datasets/tree/master/datasets)

We now define our dataset class which we can use to read the dataset and use it with a pytorch dataloader for easy sampling. 

We first open the file and read all the data.

Each text character will be represented by a unique number so we first need all the unique characters in our text. Once we have this, we create a dictionary which maps from a unique number to a letter. After defining the reverse mapping aswell, we use the dictionary to convert our original string into a list of numbers where each number represents a text character.

The labels are simply the input but shifted by one as we are always predicting the next character based on the current one.

In [2]:
class CharRNNDataset():
    def __init__(self, txt_file_path='Data/lyrics.txt', chunk_size=100, transform=None):
        self.txt_file_path = txt_file_path
        self.chunk_size = chunk_size
        self.transform = transform
        
        #open our text file and read all the data into the rawtxt variable
        with open('lyrics.txt', 'r') as file:
            rawtxt = file.read()

        #turn all of the text into lowercase as it will reduce the number of characters that our algorithm needs to learn
        rawtxt = rawtxt.lower()
        
        letters = set(rawtxt) #returns the list of unique characters in our raw text
        self.nchars = len(letters) #number of unique characters in our text file
        self.num_to_let = dict(enumerate(letters)) #created the dictionary mapping
        self.let_to_num = dict(zip(self.num_to_let.values(), self.num_to_let.keys())) #create the reverse mapping so we can map from a character to a unique number
        
        txt = list(rawtxt)#convert string to list
        #iterate through our text and change the value for each character to its mapped value
        for k, letter in enumerate(txt):
            txt[k] = self.let_to_num[letter]

        self.X = np.array(txt)
    
    def __len__(self):
        return len(self.X)-1-self.chunk_size
    
    def __getitem__(self, idx):
        x = self.X[idx:idx+self.chunk_size]
        y = self.X[idx+1:idx+self.chunk_size+1]
        
        if self.transform:
            x, y = self.transform((x, y))
    
        return x, y

In [3]:
from torchvision import transforms
class ToLongTensor():
    def __init__(self):
        pass
    def __call__(self, inp):
        return (torch.LongTensor(var) for var in inp)

In [4]:
from torch.utils.data import DataLoader

batch_size = 32
chunk_size = 100 #the length of the sequences which we will optimize over

train_data = CharRNNDataset('lyrics.txt', chunk_size=100, transform=ToLongTensor())
x, y = train_data[0]
print('First input', x)
print('First label', y, '\n')

nchars = train_data.nchars
num_to_let = train_data.num_to_let
let_to_num = train_data.let_to_num

print('Number of unique chatacters:', nchars)
print('Length of dataset:', len(train_data))

train_loader = DataLoader(train_data,# make the training dataloader
                          batch_size = batch_size,
                          shuffle=True)

First input tensor([50, 15, 18, 53, 44, 38, 44, 52, 18, 51, 18, 51, 10, 18, 52, 44,  6, 15,
        52, 54, 26, 44,  6, 50,  5, 12, 21, 38, 25, 27, 18,  6, 44, 50,  5, 12,
        44, 25, 52, 38, 51, 18, 44, 50, 46, 46,  3, 21, 50,  5, 25, 18,  6, 55,
        34, 38,  5, 18,  6,  6, 18, 44, 50, 44,  5, 38, 16, 16, 50, 44, 21, 38,
        19, 27, 44,  6,  3, 51, 18, 44, 25,  3, 54,  5, 19, 18, 52, 34, 18, 38,
        19,  6, 55, 10, 54, 19, 44,  5,  3, 21])
First label tensor([15, 18, 53, 44, 38, 44, 52, 18, 51, 18, 51, 10, 18, 52, 44,  6, 15, 52,
        54, 26, 44,  6, 50,  5, 12, 21, 38, 25, 27, 18,  6, 44, 50,  5, 12, 44,
        25, 52, 38, 51, 18, 44, 50, 46, 46,  3, 21, 50,  5, 25, 18,  6, 55, 34,
        38,  5, 18,  6,  6, 18, 44, 50, 44,  5, 38, 16, 16, 50, 44, 21, 38, 19,
        27, 44,  6,  3, 51, 18, 44, 25,  3, 54,  5, 19, 18, 52, 34, 18, 38, 19,
         6, 55, 10, 54, 19, 44,  5,  3, 21, 44]) 

Number of unique chatacters: 56
Length of dataset: 24341


Define our model which takes in variables defining its structure as parameters. The encoder converts each unique number into an embedding which is fed into the rnn model. The RNN calculates the hidden state which is converted into an output through a fully connected layer called the decoder.<br>
We also define the init_hidden function which outputs us a tensor of zeros of the required size for the hidden state.

In [5]:
class CharRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.encoder = torch.nn.Embedding(input_size, input_size)
        self.i2h = torch.nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedding = self.encoder(x) #encode our input into a vector embedding
        combined = torch.cat((embedding, hidden), 1)
        hidden = torch.tanh(self.i2h(combined))
        output = self.i2o(hidden)
        output = F.log_softmax(output, dim=1)
        return output, hidden

    def init_hidden(self, x):
        return torch.zeros(x.shape[0], self.hidden_size)

Instantiate our model, define the appropriate hyper-parameters, cost function and optimizer. We will be training on ranom samples from the text of length chunk_size so it is what batch size is to normal neural networks.

In [6]:
#hyper-params
lr = 0.001
epochs = 50

myrnn = CharRNN(nchars, 512, nchars) #instantiate our model from the class defined earlier
criterion = torch.nn.NLLLoss() #define our cost function
optimizer = torch.optim.Adam(myrnn.parameters(), lr=lr) #choose optimizer

# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # we will use this to show our models performance on a graph

Define the training loop, sequentially feeding in multiple batches of random chunks of text, summing the cost for each character in the sequence (backpropagation through time) and calculating the gradients to update our weights.

In [ ]:
#training loop
def train(model, epochs):
    for epoch in range(epochs):
        epoch_loss = 0 #stores the cost per epoch
        generatedstrings = ['']*batch_size #stores the text generated by our model each epoch
        #given our chunk size, how many chunks do we need to optimize over to have gone thorough our whole dataset
        for idx, (x, y) in enumerate(train_loader):
            loss = 0 #cost for this chunk
            h = model.init_hidden(x) #initialize our hidden state to 0s
            #sequentially input each character in our sequence and calculate loss
            for i in range(chunk_size):
                out, h = myrnn.forward(x[:, i], h) #calculate outputs based on input and previous hidden state
                
                #based on our output, what character does our network predict is next?
                #print(out)
                _, outl = out.data.max(1)
                for k in range(len(outl)):
                    letter = num_to_let[outl[k].item()]
                    generatedstrings[k]+=letter #add the predicted letter to our generated sequence
                loss += criterion(out, y[:, i]) #add the cost for this input to the cost for this current chunk
            
            writer.add_scalar('Loss/Train', loss/chunk_size, epoch*len(train_loader) + idx)    # write loss to a graph
            
            #based on the sum of the cost for this sequence (backpropagation through time) calculate the gradients and update our weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss+=loss.item() #add the cost of this sequence to the cost of this epoch
        epoch_loss /= len(train_loader.dataset) #divide by the number of chunks per epoch to get average cost per epoch

        print('Epoch ', epoch+1, ' Avg Loss: ', epoch_loss)
        print('Generated text: ', generatedstrings[np.random.randint(0, batch_size)][0:750], '\n')
        
train(myrnn, epochs)

The generated text above picks the most probable next character each time. This is not the best way to do it as our model will be deterministic so it will produce the same text over and over again. To get it producing different text, we should instead sample from the probability distribution of possible next letters output by the network. That is what we will do with the next function.

In [ ]:
def maparray(txt, mapdict):
    txt = list(txt)
    #iterate through our text and change the value for each character to its mapped value
    for k, letter in enumerate(txt):
        txt[k] = mapdict[letter]
    txt = np.array(txt)
    return txt

def generate(model, prime_str='a', str_len=150, temperature=0.75):
    generated = prime_str
    
    prime_str = maparray(prime_str, let_to_num)
    x = torch.LongTensor(prime_str).unsqueeze(0)
    
    #initialize hidden state
    h = model.init_hidden(x)
    
    #primes our hidden state with the input string
    for i in range(x.shape[1]):
        out, h = model.forward(x[:, i], h)
    
    x = x[:, -1]
    for i in range(str_len):
        out, h = myrnn.forward(x, h)
        
        out_dist = out.data.view(-1).div(temperature).exp()
        sample = torch.multinomial(out_dist, 1).item()
        pred_char = num_to_let[sample]
        
        generated += pred_char
        
        x = torch.LongTensor([sample])
    
    return generated
        
gen = generate(myrnn, 'this be ', 2000, 0.75)
print(gen)

In [18]:
class CharRNN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super().__init__()
        #store input parameters in the object so we can use them later on
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        #required functions for model
        self.encoder = torch.nn.Embedding(input_size, input_size)
        self.rnn = torch.nn.RNN(input_size, hidden_size, n_layers, batch_first=True)
        self.decoder = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedding = self.encoder(x.view(-1)) #encode our input into a vector embedding
        output, hidden = self.rnn(embedding.view(-1, 1, self.input_size), hidden) #calculate the output from our rnn based on our input and previous hidden state
        output = self.decoder(output.view(-1, self.hidden_size)) #calculate our output based on output of rnn
        output = F.log_softmax(output, dim=1)
        return output, hidden

    def init_hidden(self, x):
        return torch.zeros(self.n_layers, x.shape[0], self.hidden_size) #initialize our hidden state to a matrix of 0s

In [19]:
#hyper-params
lr = 0.001
epochs = 50

myrnn = CharRNN(nchars, 512, nchars) #instantiate our model from the class defined earlier
criterion = torch.nn.NLLLoss() #define our cost function
optimizer = torch.optim.Adam(myrnn.parameters(), lr=lr) #choose optimizer

# SET UP TRAINING VISUALISATION
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # we will use this to show our models performance on a graph

In [ ]:
train(myrnn, epochs)

In [21]:
gen = generate(myrnn, 'this be ', 2000, 0.75)
print(gen)

this be years
and forkid."t, coll shit without a bid freat spick off me, byouch tas aye
you do not amaze me, aye
i blew me away, you think more beauty in blue green and grey
all my solomon up north, 12 years a slave
12 years of age, thinkin’ my shade tho sheterr
dount don't want to self-destruct
the evils of lucy was all around me
so i well of me marain took my stage, i'm the sandman (sandman)
get the fuck off my stage, i'm the sandman (sandman)
get the fuck off my stage, i'm the sandman (sandman)
get the fuck off my stage, i'm the realest nigga, my fiers, new, i got winners on the way
you ain't never gotta say shit, uh
and i know your taste is, a little bit hmm, high maintenance, uh
everybody else basic, you live life on an everyday basis
with poetic justice, poetic justice
if i told you that a flower bloomed in a dark room, would you trust it?
i mean you need to hear this, love is not just a verb
it's you looking in the mirror, love is not just a verb
it's you looking in the mirror, 